In [2]:
import json, warnings, os
from pathlib import Path

warnings.filterwarnings('ignore')

if Path.cwd().name == "notebooks":
    rootdir = Path.cwd().parent
else:
    rootdir = Path.cwd()

import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

from stables.utils.postgres import get_sqlalchemy_engine
from stables.config import local_pg_config, remote_pg_config
engine = get_sqlalchemy_engine(local_pg_config)

## TVL

### TVL by stablecoins

In [2]:
latest_tvl_query = """
WITH latest_date AS (
    SELECT MAX(DATE(time)) as max_date
    FROM llama.circulating
)
SELECT 
    (SELECT max_date FROM latest_date) as latest_date,
    SUM(CASE WHEN id = 146 THEN circulating ELSE 0 END) as tvl_usde,
    SUM(CASE WHEN id = 221 THEN circulating ELSE 0 END) as tvl_usdtb
FROM llama.circulating
WHERE DATE(time) = (SELECT max_date FROM latest_date)
"""
latest_tvl_df = pd.read_sql(latest_tvl_query, engine)
latest_date = latest_tvl_df['latest_date'].iloc[-1]
tvl_usde = latest_tvl_df['tvl_usde'].iloc[-1]
tvl_usdtb = latest_tvl_df['tvl_usdtb'].iloc[-1]
tvl = tvl_usde + tvl_usdtb


grouped_query = """
WITH data AS (
    SELECT 
        DATE(time) as date,
        CASE 
            WHEN id = 146 THEN 'USDe'
        WHEN id = 221 THEN 'USDTB'
        ELSE CAST(id AS VARCHAR)
    END as id,
    SUM(circulating) as tvl
    FROM llama.circulating
    GROUP BY DATE(time), id
    ORDER BY DATE(time), id
),
sorted_data AS (
    SELECT 
        date,
        id,
        tvl,
        ROW_NUMBER() OVER (PARTITION BY date, id ORDER BY date DESC) as rn
    FROM data
)
SELECT 
    date,
    id,
    tvl
FROM sorted_data
WHERE rn = 1
ORDER BY date, id
"""
df = pd.read_sql(grouped_query, engine)

fig = px.bar(df, x="date", y="tvl", color="id", title=f"Current TVL: {tvl/1e9:.2f}B, USDe: {tvl_usde/1e9:.2f}B, USDTB: {tvl_usdtb/1e9:.2f}B",labels="")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Stablecoin"
)
fig.show()

### TVL by chains

In [3]:
query = """
WITH data AS (
    SELECT 
        DATE(time) as date,
        chain,
        SUM(circulating) as tvl
    FROM llama.circulating
    GROUP BY DATE(time), chain
),
sorted_data AS (
    SELECT 
        date,
        chain,
        tvl,
        ROW_NUMBER() OVER (PARTITION BY date, chain ORDER BY date DESC) as rn
    FROM data
)
SELECT 
    date,
    chain,
    tvl
FROM sorted_data
WHERE rn = 1
ORDER BY date, chain
"""

df = pd.read_sql(query, engine)

fig = px.bar(df, x="date", y="tvl", color="chain", title="Ethena TVL by chains")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Chain"
)
fig.show()

## USDe

### TVL


In [3]:
query = """
SELECT * FROM llama.circulating
WHERE id = 146
"""
df= pd.read_sql(query, engine)

l = df['time'].max()

tvl = df[(df['time'] == l)]['circulating'].sum()

n_chains = df[
    (df['time'] == l) 
]['chain'].nunique()


df = df.groupby(['time', "chain"]).agg(
    tvl=('circulating', 'sum')
).reset_index()
df

fig = px.bar(df, x="time", y="tvl", color="chain", title=f"{l.date()}, TVL: {tvl/1e9:.2f}B", )
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Chain"
)
fig.show()

### Primary market
Mint and Redeem

In [4]:
query = """
WITH combined_events AS (
    SELECT
        contract_address,
        block_timestamp,
        benefactor,
        beneficiary,
        caller,
        collateral_asset,
        collateral_amount,
        event_type,
        CASE 
            WHEN event_type = 'mint' THEN usde_amount 
            ELSE -usde_amount 
        END as usde_amount
    FROM ethena_.usde_mint_redeem_v1_events

    UNION ALL

    SELECT 
        contract_address,
        block_timestamp,
        benefactor,
        beneficiary,
        caller,
        collateral_asset,
        collateral_amount,
        event_type,
        CASE 
            WHEN event_type = 'mint' THEN usde_amount 
            ELSE -usde_amount 
        END as usde_amount
    FROM ethena_.usde_mint_redeem_v2_events
),

daily_totals AS (
    SELECT 
        DATE(block_timestamp) as date,
        event_type,
        SUM(usde_amount) as usde_amount
    FROM combined_events
    GROUP BY DATE(block_timestamp), event_type
),

daily_pivot AS (
    SELECT 
        date,
        SUM(CASE WHEN event_type = 'mint' THEN usde_amount ELSE 0 END) as mint,
        SUM(CASE WHEN event_type = 'redeem' THEN usde_amount ELSE 0 END) as redeem,
        SUM(usde_amount) as net_change
    FROM daily_totals
    GROUP BY date
)

SELECT 
    date,
    mint / 1e18 as mint,
    redeem / 1e18 as redeem,
    net_change / 1e18 as net_change
FROM daily_pivot
ORDER BY date;
"""
df= pd.read_sql(query, engine)

# Create the plot
fig = go.Figure()

# Add mint bars
fig.add_trace(go.Bar(
    x=df.index,
    y=df['mint'],
    name='Mint',
    marker_color='green',
    # opacity=0.7
))

# Add redeem bars
fig.add_trace(go.Bar(
    x=df.index,
    y=df['redeem'],
    name='Redeem',
    marker_color='red',
    # opacity=0.7
))

# Add net change line
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['net_change'],
    name='Net Change',
    mode='lines',
    line=dict(color='blue', width=1),
    # marker=dict(size=2),
    opacity=0.7
))

# Update layout
fig.update_layout(
    title='Daily USDE Mint/Redeem Activities',
    xaxis_title='Date',
    yaxis_title='Amount (USDE)',
    barmode='group',
    hovermode='x unified'
)

# Show the plot
fig.show() 

### USDe staking

In [6]:
query = """
WITH usde_data AS (
    SELECT 
        DATE(time) as date,
        SUM(circulating) as usde
    FROM llama.circulating
    WHERE id = 146
    GROUP BY DATE(time)
),
susde_data AS (
    SELECT 
        DATE(time) as date,
        tvl_usd as susde
    FROM llama.yield_pools
    WHERE pool_id = '66985a81-9c51-46ca-9977-42b4fe7bc6df'
),
combined_data AS (
    SELECT 
        COALESCE(u.date, s.date) as date,
        u.usde,
        s.susde
    FROM usde_data u
    FULL OUTER JOIN susde_data s ON u.date = s.date
),
ranked_data AS (
    SELECT 
        date,
        usde,
        susde,
        CASE 
            WHEN usde IS NOT NULL AND usde > 0 
            THEN (susde / usde) * 100 
            ELSE NULL 
        END as staking_perc,
        ROW_NUMBER() OVER (PARTITION BY date ORDER BY date DESC) as rn
    FROM combined_data
)
SELECT 
    date,
    usde,
    susde,
    staking_perc
FROM ranked_data
WHERE rn = 1
ORDER BY date
"""
df = pd.read_sql(query, engine)
current_staking_perc = df[df['date'] == df['date'].max()]['staking_perc'].values[0]

fig = go.Figure()
fig.add_trace(go.Bar(x=df['date'], y=df['usde'], name='USDe', opacity=0.5, yaxis='y'))

fig.add_trace(go.Scatter(
      x=df['date'],
      y=df['staking_perc'],
      mode='lines',
      name='Staking %',
      yaxis='y2'
  ))
fig.update_layout(
      yaxis2=dict(
          title="%",
          overlaying='y',
          side='right'
      ),
    # title="Current USDe TVL: {:.2f}B, staking {:.2f}%".format(tvl_usde/1e9, current_staking_perc),
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
  )